# Finding adversarial examples, in depth

In the quickstart, we used the default parameters for `find_adversarial_example`. Using the same example from the quickstart, we explore how to get more out of the function.

In [1]:
using MIPVerify
using Gurobi
using JuMP
using Images
using Printf

mnist = MIPVerify.read_datasets("MNIST")
n1 = MIPVerify.get_example_network_params("MNIST.n1")
sample_image = MIPVerify.get_image(mnist.test.images, 1);

function print_summary(d::Dict)
    # Helper function to print out output
    obj_val = JuMP.objective_value(d[:Model])
    solve_time = JuMP.solve_time(d[:Model])
    println("Objective Value: $(@sprintf("%.6f", obj_val)), Solve Time: $(@sprintf("%.2f", solve_time))")
end

function view_diff(diff::Array{<:Real, 2})
    n = 1001
    colormap("RdBu", n)[ceil.(Int, (diff+1)/2*n)]
end

view_diff (generic function with 1 method)

## `find_adversarial_example`

`find_adversarial_example` takes five positional arguments

```
find_adversarial_example(nn, input, target_selection, optimizer, main_solve_options)
```

It also takes named arguments, each with the default value specified.

```
norm_order = 1
invert_target_selection = false
pp = MIPVerify.UnrestrictedPerturbationFamily()
tightening_algorithm = mip
tightening_options: same as main solver, but with output suppressed and a time limit of 20s per solve.
```

See [full documentation here](https://vtjeng.github.io/MIPVerify.jl/dev/finding_adversarial_examples/single_image/#MIPVerify.find_adversarial_example-Tuple{NeuralNet,Array{#s165,N}%20where%20N%20where%20#s165%3C:Real,Union{Integer,%20Array{#s164,1}%20where%20#s164%3C:Integer},Any,Dict})

We explore what each of these options allow us to do.

# Basic Options

## Specifying target categories for the adversarial example

`target_selection` and `invert_target_selection` control what the category we want the adversarial example to be classified in.

**Specification**: `target_selection` accepts either a single integer or a list of integers.

For example, if I wanted the original image (which is the digit 7) to be classified as the digit 8 or 9, I could run two separate solves with `target_selection=9` and `target_selection=10` (Julia is 1-indexed), finding closest adversarial examples at distance `13.763` and `4.642` ...

In [ ]:
d = MIPVerify.find_adversarial_example(
    n1, 
    sample_image, 
    9, 
    Gurobi.Optimizer,
    # OutputFlag=0 prevents any output from being printed out
    Dict(),
    pp = MIPVerify.LInfNormBoundedPerturbationFamily(0.2),
    tightening_algorithm = lp,
)
print_summary(d)

Academic license - for non-commercial use only
[notice | MIPVerify]: Attempting to find adversarial example. Neural net predicted label is 8, target labels are [9]
[notice | MIPVerify]: Determining upper and lower bounds for the input to each non-linear unit.


  Calculating upper bounds:  35%|████████               |  ETA: 0:00:00

Academic license - for non-commercial use only
Academic license - for non-commercial use only


  Calculating upper bounds: 100%|███████████████████████| Time: 0:00:00
  Calculating lower bounds: 100%|███████████████████████| Time: 0:00:00


In [ ]:
d = MIPVerify.find_adversarial_example(
    n1, 
    sample_image, 
    9, 
    Gurobi.Optimizer,
    Dict("OutputFlag" => 0),
    pp = MIPVerify.LInfNormBoundedPerturbationFamily(0.1),
    tightening_algorithm = lp,
)
print_summary(d)

Or I can can pass the targets as  `target_selection = [9, 10]`, where the same optimal value of `4.642` is found.

Solve times for multiple target labels are typically on par with or faster than the aggregate solve times when solving with each target label in sequence.

In [ ]:
d = MIPVerify.find_adversarial_example(
    n1, 
    sample_image, 
    [9, 10], 
    Gurobi.Optimizer,
    Dict("OutputFlag" => 0),
    pp = MIPVerify.LInfNormBoundedPerturbationFamily(0.1),
    tightening_algorithm = lp,
)
print_summary(d)

A common use case is to have the adversarial example being in any category but the original:

In [ ]:
d = MIPVerify.find_adversarial_example(
    n1, 
    sample_image, 
    [1, 2, 3, 4, 5, 6, 7, 9, 10], 
    Gurobi.Optimizer,
    Dict("OutputFlag" => 0),
    pp = MIPVerify.LInfNormBoundedPerturbationFamily(0.1),
    tightening_algorithm = lp,
)
print_summary(d)

Rather than typing the full list of other categories, we can set `target_selection = 8`, and `invert_target_selection = true`.

In [ ]:
d = MIPVerify.find_adversarial_example(
    n1, 
    sample_image, 
    8, 
    Gurobi.Optimizer,
    Dict("OutputFlag" => 0),
    pp = MIPVerify.LInfNormBoundedPerturbationFamily(0.1),
    tightening_algorithm = lp,
    invert_target_selection = True,
)
print_summary(d)

## Restricting the Family of Perturbations

### Unrestricted Perturbations

The standard threat model is to allow each pixel to be perturbed independently, which is what happens by default:

In [ ]:
d = @time MIPVerify.find_adversarial_example(
    n1, 
    sample_image, 
    10, 
    Gurobi.Optimizer,
    Dict("OutputFlag" => 0),
    tightening_algorithm = lp,
)
print_summary(d)
perturbed_sample_image = JuMP.value.(d[:PerturbedInput])
colorview(Gray, perturbed_sample_image[1, :, :, 1])

### $L_\infty$-norm Bounded Perturbations

We can bound the $L_\infty$-norm of the perturbation.

As long as the size of the $L_\infty$-norm bound we choose is larger than the actual ($L_\infty$-)minimal perturbation, we will find the same result, and often more quickly.

In [ ]:
d = @time MIPVerify.find_adversarial_example(
    n1, 
    sample_image, 
    10,
    Gurobi.Optimizer,
    Dict("OutputFlag" => 0),
    tightening_algorithm = lp,
    norm_order=Inf,
)
perturbed_sample_image = JuMP.value.(d[:PerturbedInput])
colorview(Gray, perturbed_sample_image[1, :, :, 1])

In [ ]:
d = @time MIPVerify.find_adversarial_example(
    n1, 
    sample_image, 
    10,
    Gurobi.Optimizer,
    Dict("OutputFlag" => 0),
    tightening_algorithm = lp,
    norm_order=Inf,
    pp = MIPVerify.LInfNormBoundedPerturbationFamily(0.05)
)
perturbed_sample_image = JuMP.value.(d[:PerturbedInput])
colorview(Gray, perturbed_sample_image[1, :, :, 1])

If the $L_\infty$-norm bound you choose is smaller than the actual minimal perturbation, the problem is Infeasible.

In [ ]:
d = @time MIPVerify.find_adversarial_example(
    n1, 
    sample_image, 
    10,
    Gurobi.Optimizer,
    Dict("OutputFlag" => 0),
    tightening_algorithm = lp,
    norm_order=Inf,
    pp = MIPVerify.LInfNormBoundedPerturbationFamily(0.03)
)

In [ ]:
d[:SolveStatus]

### Blurring Perturbations

We can restrict the perturbations to a blur with a 5x5 kernel instead. (We are still minimizing over the norm of the perturbation.)

In [ ]:
d = @time MIPVerify.find_adversarial_example(
    n1, 
    sample_image, 
    10,
    Gurobi.Optimizer,
    Dict("OutputFlag" => 0),
    tightening_algorithm = lp,
    norm_order=Inf,
    pp = MIPVerify.BlurringPerturbationFamily((5, 5))
)
perturbed_sample_image = JuMP.value.(d[:PerturbedInput])
colorview(Gray, perturbed_sample_image[1, :, :, 1])

In [ ]:
diff = JuMP.value.(d[:Perturbation])
view_diff(diff[1, :, :, 1])

## Minimizing Over Different Norms
### $l_1$
By default, we minimize the $l_1$ norm of the perturbation. This generally encourages sparsity in the perturbations. 

In this case, the minimum $l_1$ norm perturbation required for the image to be classified as a `9` is `4.641859.`

In [ ]:
d = MIPVerify.find_adversarial_example(
    n1, 
    sample_image, 
    10,
    Gurobi.Optimizer,
    Dict("OutputFlag" => 0),
    tightening_algorithm = lp,
    norm_order=Inf
)
print_summary(d)
perturbed_sample_image = JuMP.value.(d[:PerturbedInput])
colorview(Gray, perturbed_sample_image[1, :, :, 1])

We also show the difference between the perturbed image and the original image. Red is areas of decreased brightness and blue is areas of increased brightness.

In [ ]:
diff = getvalue(d[:Perturbation])
view_diff(diff[1, :, :, 1])

### $l_\infty$

We can also minimize over the $l_\infty$ norm. This generally results in large patches of the image being changed. 

In this case, the minimum $l_\infty$ norm perturbation required for the image to be classified as a `9` is `0.046085.`

In [ ]:
d = MIPVerify.find_adversarial_example(n1, sample_image, 10, GurobiSolver(OutputFlag=0),
    norm_order=Inf);
print_summary(d)
perturbed_sample_image = getvalue(d[:PerturbedInput])
colorview(Gray, perturbed_sample_image[1, :, :, 1])

In [ ]:
diff = getvalue(d[:Perturbation])
view_diff(diff[1, :, :, 1])

### $l_2$
With solvers that can handle MIQPs (like Gurobi), we can minimize over the $l_2$ norm. This generally takes a bit more time. 

In this case, the minimum $l_2$ norm perturbation required for the image to be classified as a `9` is `0.705367 = sqrt(0.497542).`

In [ ]:
d = MIPVerify.find_adversarial_example(n1, sample_image, 10, GurobiSolver(OutputFlag=0),
    norm_order=2);
print_summary(d)
perturbed_sample_image = getvalue(d[:PerturbedInput])
colorview(Gray, perturbed_sample_image[1, :, :, 1])

In [ ]:
diff = getvalue(d[:Perturbation])
view_diff(diff[1, :, :, 1])

# Advanced Options

### `tightening_algorithm`

By default, we tighten the bounds on each intermediate value by solving an MIP using the `tightening_solver`. Compare total solve times for three different tightening algorithms:

In [ ]:
@time MIPVerify.find_adversarial_example(n1, sample_image, 10, GurobiSolver(OutputFlag=0), rebuild=true,
    tightening_algorithm=interval_arithmetic);

In [ ]:
@time MIPVerify.find_adversarial_example(n1, sample_image, 10, GurobiSolver(OutputFlag=0), rebuild=true,
    tightening_algorithm=lp, tightening_solver = GurobiSolver(Gurobi.Env(), OutputFlag=0));

In [ ]:
MIPVerify.setloglevel!("notice")
@time MIPVerify.find_adversarial_example(n1, sample_image, 10, GurobiSolver(OutputFlag=0), rebuild=true,
    tightening_algorithm=mip);

We can also modify many of the parameters of the solver to change behavior:

We will be focusing on the parameters available via Gurobi (http://www.gurobi.com/documentation/7.5/refman/parameters.html), but other solvers often have similar options.

  
### `main_solver`

#### Muting Output
To mute the output from the `GurobiSolver`, set `OutputFlag=0`.


In [ ]:
d = MIPVerify.find_adversarial_example(n1, sample_image, 10, GurobiSolver());
print_summary(d)

In [ ]:
MIPVerify.find_adversarial_example(n1, sample_image, 10, GurobiSolver(OutputFlag=0));
print_summary(d)

#### Terminating early if a conditon is satisfied

Sometimes, finding an adversarial example takes a long time:

In [ ]:
MIPVerify.find_adversarial_example(n1, sample_image, 9, GurobiSolver(),
    norm_order=Inf);

You may want to terminate early when a particular condition is satisfied. Common reasons are:

  1. Solve exceeding time limit
  2. Lower bound on robustness proved (i.e. `BestBd` increases above a pre-determined threshold)
  3. Counter-example found (i.e. `Incumbent` adversarial image found that is closer to the original image than expected).
  4. Difference between `Incumbent` and `BestBd` falls below a pre-determined threshold.
  
Fortunately, Gurobi has a parameter for all of these cases.

##### Terminate if time limit is reached
Set `TimeLimit`:

In [ ]:
MIPVerify.find_adversarial_example(n1, sample_image, 9, GurobiSolver(TimeLimit=30),
    norm_order=Inf);

##### Terminate if lower bound on robustness proved

Set `BestBdStop` or `Cutoff`.

(`Cutoff` gives a different error message that is not currently processed correctly by the latest release of `Gurobi.jl`).

In [ ]:
MIPVerify.find_adversarial_example(n1, sample_image, 9, GurobiSolver(BestBdStop=0.05),
    norm_order=Inf);

##### Terminate if adversarial example found closer than expected robustness

Set `BestObjStop`.

In [ ]:
MIPVerify.find_adversarial_example(n1, sample_image, 9, GurobiSolver(BestObjStop=0.2),
    norm_order=Inf);

##### Terminate if gap between `Incumbent` and `BestBd` is below threshold

Set `MIPGap`.

In [ ]:
MIPVerify.find_adversarial_example(n1, sample_image, 9, GurobiSolver(MIPGap=0.4),
    norm_order=Inf);

### `tightening_solver`
The default model build solver has the same type as the `main_solver`, but uses the default settings for that solver type other than 1) muting the output and 2) setting a time limit of 20s per solve (i.e. per upper/lower bound per intermediate value).

The most common reason to pass in your own `tightening_solver` to modify the time limit per solve.

Whew! That was a lot. The next tutorial will introduce you to everything you can extract from the results dictionary.